<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bbd0273227c9050f236458221e0ba4c04c07c60f9eb3d3dffcdd3b4aec121cf5
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-06 12:25:32
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -28.67 K (-0.2%)
Current PnL: -20.74 L (-13.76%)
CY Booked + Current PnL: -9.30 L (-6.17%)
-------------------
Total profit:  1.42 L
Total loss:  -22.16 L
-------------------
Total Booked + Current PnL: 17.68 L (14.3%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.12%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.94 L (62.44%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.89%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.13,117215.0,16711.0,12401.0,3.11,16.63,10.58,28.96,79.0,1.35,0.84,56.24,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.17,38.0,M-SC,1.53,83192.0,-17585.0,17678.0,-1.38,-17.45,21.25,0.09,245.0,-0.99,0.59,9.30,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,10.09,49.0,H-MC,2.98,189920.0,25250.0,18080.0,1.14,15.33,9.52,26.31,89.0,1.40,1.35,24.54,X40N,NTT,AC
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195942.0,-4196.0,25081.0,-0.89,-2.10,12.80,10.44,4.0,-0.17,1.40,3.35,X40,NTT,FMCG
34,ICICIGI,2260.25,-17.56,57.0,H-MC,1.76,141784.0,5651.0,25479.0,0.23,4.15,17.97,22.86,91.0,0.22,1.01,17.29,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-40.28,44.0,H-SC,16.07,90540.0,-11960.0,52558.0,5.00,-11.67,58.05,39.61,114.0,-0.23,0.65,10.58,AR,ATH,MISC
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.13,117215.0,16711.0,12401.0,3.11,16.63,10.58,28.96,79.0,1.35,0.84,56.24,MH,ATH,METALS
48,LTIM,7230.20,-4.62,52.0,H-LC,1.99,236385.0,-9766.0,88975.0,2.60,-3.97,37.64,32.18,16.0,-0.11,1.69,30.65,X200,ATH,IT
4,ANGELONE,3033.00,4.08,48.0,H-SC,11.30,184262.0,-6744.0,64436.0,2.08,-3.53,34.97,30.21,157.0,-0.10,1.31,16.61,X40N,NTT,FINANCE
65,SIEMENS,4671.50,0.39,55.0,H-LC,1.91,161245.0,-24850.0,72335.0,1.92,-13.35,44.86,25.51,15.0,-0.34,1.15,18.83,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-180.68,61.0,M-SC,11.40,140121.0,-9099.0,153278.0,-5.66,-6.10,109.39,96.62,208.0,-0.06,1.00,51.64,XY25,NTT,FINANCE
66,SIS,528.00,2010.88,32.0,H-SC,3.61,85183.0,-25849.0,49457.0,-2.69,-23.28,58.06,21.26,156.0,-0.52,0.61,15.09,OX40N,NTT,MISC
24,DEN,75.00,96.89,45.0,M-SC,2.09,55296.0,-29168.0,64702.0,-1.90,-34.53,117.01,42.07,237.0,-0.45,0.39,17.43,AR,NTT,ENTERTAINMENT
13,BERGEPAINT,680.00,-19.75,49.0,H-MC,3.46,215662.0,-11677.0,59738.0,-1.86,-5.14,27.70,21.14,106.0,-0.20,1.54,21.56,XY24,NTT,PAINTS
46,KPIGREEN,731.05,-5.95,33.0,H-SC,7.36,111964.0,-13333.0,72262.0,-1.83,-10.64,64.54,47.03,141.0,-0.18,0.80,40.19,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.34,47.0,H-MC,2.14,135385.0,-403.0,43161.0,-0.34,-0.3,31.88,31.48,107.0,-0.01,0.97,11.79,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.70,43.0,M-LC,5.66,150046.0,-899.0,110359.0,0.43,-0.6,73.55,72.51,53.0,-0.01,1.07,5.39,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,67.0,H-SC,12.87,131679.0,-10800.0,30326.0,0.89,-7.58,23.03,13.70,163.0,-0.36,0.94,55.42,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.24,57.0,H-SC,2.01,225531.0,-44136.0,80469.0,-0.70,-16.37,35.68,13.47,138.0,-0.55,1.61,14.68,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,38.0,M-SC,1.53,83192.0,-17585.0,17678.0,-1.38,-17.45,21.25,0.09,245.0,-0.99,0.59,9.30,OX40N,NTT,DURABLES
18,CERA,9475.0,-23.15,36.0,H-SC,1.48,140070.0,-35833.0,77851.0,-0.95,-20.37,55.58,23.89,149.0,-0.46,1.00,20.97,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.38,42.0,H-MC,6.24,102210.0,-29625.0,71843.0,0.45,-22.47,70.29,32.03,98.0,-0.41,0.73,14.48,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1262.04,71.0,M-SC,20.21,185239.0,4096.0,41364.0,0.39,2.26,22.33,25.10,235.0,0.10,1.32,48.45,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,12.19,232401.0,7439.0,160217.0,0.03,3.31,68.94,74.53,192.0,0.05,1.66,36.63,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,5.59,214020.0,8944.0,103628.0,0.10,4.36,48.42,54.89,52.0,0.09,1.53,29.51,X5K,ATH,METALS
14,BLUESTARCO,2080.00,10.09,49.0,H-MC,2.98,189920.0,25250.0,18080.0,1.14,15.33,9.52,26.31,89.0,1.40,1.35,24.54,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,43.0,H-MC,1.62,202485.0,10743.0,27011.0,-0.77,5.60,13.34,19.69,99.0,0.40,1.44,12.78,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.13,117215.0,16711.0,12401.0,3.11,16.63,10.58,28.96,79.0,1.35,0.84,56.24,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,68.0,H-MC,10.17,192773.0,12965.0,99452.0,-0.15,7.21,51.59,62.52,88.0,0.13,1.37,43.76,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,5.59,214020.0,8944.0,103628.0,0.10,4.36,48.42,54.89,52.0,0.09,1.53,29.51,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.01,34.0,H-SC,9.50,125615.0,2279.0,129346.0,-0.06,1.85,102.97,106.72,119.0,0.02,0.90,25.66,AR,ATH,MISC
86,WIPRO,420.00,-14.70,43.0,M-LC,5.66,150046.0,-899.0,110359.0,0.43,-0.60,73.55,72.51,53.0,-0.01,1.07,5.39,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.42,31.0,H-LC,5.80,205762.0,-46006.0,99218.0,-0.83,-18.27,48.22,21.13,27.0,-0.46,1.47,10.68,X40,ATH,PAINTS
74,TCS,4406.34,-31.58,31.0,H-LC,12.33,272779.0,-73177.0,141409.0,-0.42,-21.15,51.84,19.73,1.0,-0.52,1.94,0.47,X40,ATH,IT
83,VBL,671.64,-21.40,32.0,H-LC,9.62,282921.0,-32921.0,145591.0,0.11,-10.42,51.46,35.67,5.0,-0.23,2.02,1.99,X40N,ATH,FMCG
19,CIPLA,1795.00,-22.52,41.0,H-LC,4.00,206308.0,1808.0,39611.0,-0.78,0.88,19.20,20.25,10.0,0.05,1.47,9.91,X40N,BTT,PHARMA
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195942.0,-4196.0,25081.0,-0.89,-2.10,12.80,10.44,4.0,-0.17,1.40,3.35,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.39,55.0,H-LC,1.91,161245.0,-24850.0,72335.0,1.92,-13.35,44.86,25.51,15.0,-0.34,1.15,18.83,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-4.62,52.0,H-LC,1.99,236385.0,-9766.0,88975.0,2.60,-3.97,37.64,32.18,16.0,-0.11,1.69,30.65,X200,ATH,IT
42,ITC,452.0,-39.19,41.0,H-LC,2.04,195942.0,-4196.0,25081.0,-0.89,-2.10,12.80,10.44,4.0,-0.17,1.40,3.35,X40,NTT,FMCG
51,NESTLEIND,1377.0,-10.77,49.0,H-LC,3.20,274763.0,9337.0,47452.0,0.37,3.52,17.27,21.40,11.0,0.20,1.96,10.39,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.79,41.0,H-LC,3.39,212971.0,2605.0,26174.0,0.13,1.24,12.29,13.68,37.0,0.10,1.52,17.97,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-31.58,31.0,H-LC,12.33,272779.0,-73177.0,141409.0,-0.42,-21.15,51.84,19.73,1.0,-0.52,1.94,0.47,X40,ATH,IT
83,VBL,671.64,-21.40,32.0,H-LC,9.62,282921.0,-32921.0,145591.0,0.11,-10.42,51.46,35.67,5.0,-0.23,2.02,1.99,X40N,ATH,FMCG
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195942.0,-4196.0,25081.0,-0.89,-2.10,12.80,10.44,4.0,-0.17,1.40,3.35,X40,NTT,FMCG
30,HCLTECH,1937.36,-2.37,44.0,H-LC,10.00,222469.0,-19455.0,81691.0,1.69,-8.04,36.72,25.73,8.0,-0.24,1.59,5.13,X40,ATH,IT
80,UNITDSPR,1644.00,-11.37,56.0,H-LC,7.00,232389.0,-1755.0,50382.0,-0.84,-0.75,21.68,20.77,86.0,-0.03,1.66,6.12,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.39,55.0,H-LC,1.91,161245.0,-24850.0,72335.0,1.92,-13.35,44.86,25.51,15.0,-0.34,1.15,18.83,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195942.0,-4196.0,25081.0,-0.89,-2.10,12.80,10.44,4.0,-0.17,1.40,3.35,X40,NTT,FMCG
19,CIPLA,1795.00,-22.52,41.0,H-LC,4.00,206308.0,1808.0,39611.0,-0.78,0.88,19.20,20.25,10.0,0.05,1.47,9.91,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.42,31.0,H-LC,5.80,205762.0,-46006.0,99218.0,-0.83,-18.27,48.22,21.13,27.0,-0.46,1.47,10.68,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,41.0,H-LC,3.39,212971.0,2605.0,26174.0,0.13,1.24,12.29,13.68,37.0,0.10,1.52,17.97,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-31.58,31.0,H-LC,12.33,272779.0,-73177.0,141409.0,-0.42,-21.15,51.84,19.73,1.0,-0.52,1.94,0.47,X40,ATH,IT
40,INFY,2275.00,-19.96,45.0,H-LC,8.15,311512.0,-1376.0,173076.0,1.10,-0.44,55.56,54.87,3.0,-0.01,2.22,6.18,X40,BTT,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195942.0,-4196.0,25081.0,-0.89,-2.10,12.80,10.44,4.0,-0.17,1.40,3.35,X40,NTT,FMCG
83,VBL,671.64,-21.40,32.0,H-LC,9.62,282921.0,-32921.0,145591.0,0.11,-10.42,51.46,35.67,5.0,-0.23,2.02,1.99,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,47.0,H-LC,7.57,249600.0,-12019.0,131240.0,0.32,-4.59,52.58,45.57,7.0,-0.09,1.78,6.21,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,32.80,79584.0,-33965.0,74013.0,0.53,-29.91,93.00,35.27,268.0,-0.46,0.57,94.78,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,46.0,H-SC,11.50,92040.0,-5940.0,27548.0,0.99,-6.06,29.93,22.05,152.0,-0.22,0.66,33.79,XR,ATH,FINANCE
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.13,117215.0,16711.0,12401.0,3.11,16.63,10.58,28.96,79.0,1.35,0.84,56.24,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,67.0,H-SC,12.87,131679.0,-10800.0,30326.0,0.89,-7.58,23.03,13.70,163.0,-0.36,0.94,55.42,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-180.68,61.0,M-SC,11.40,140121.0,-9099.0,153278.0,-5.66,-6.10,109.39,96.62,208.0,-0.06,1.00,51.64,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,66.67,68.0,H-SC,5.98,152429.0,-26642.0,92372.0,-1.68,-14.88,60.60,36.71,135.0,-0.29,1.09,51.24,XR,NTT,BANKS
64,SHALBY,327.00,1262.04,71.0,M-SC,20.21,185239.0,4096.0,41364.0,0.39,2.26,22.33,25.10,235.0,0.10,1.32,48.45,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-180.68,61.0,M-SC,11.40,140121.0,-9099.0,153278.0,-5.66,-6.10,109.39,96.62,208.0,-0.06,1.00,51.64,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.07,68.0,M-LC,5.59,214020.0,8944.0,103628.0,0.10,4.36,48.42,54.89,52.0,0.09,1.53,29.51,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,68.0,H-MC,10.17,192773.0,12965.0,99452.0,-0.15,7.21,51.59,62.52,88.0,0.13,1.37,43.76,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.00
1,25,43.27
2,50,74.93


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.22
LC    32.09
MC    23.75
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      14.52
X40N     11.22
XR       11.04
XY25     10.32
AR        8.24
OX40N     7.75
X200      1.69
MH        1.64
X5K       1.53
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.12
H-LC    25.54
H-MC    20.47
M-SC    13.67
M-LC     5.54
M-MC     2.98
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.81,-6.03,40.98
IT,12.62,-18.93,83.34
FINANCE,10.43,-15.25,67.60
MISC,6.96,-17.79,83.06
BANKS,6.39,-11.36,69.94
PAINTS,5.56,-18.50,36.16
ELECTRICAL,5.35,-9.76,48.27
INSURANCE,3.76,-5.04,44.31
AUTO,3.53,-16.13,71.65


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3231212.0,22
XR,1361953.0,14
AR,1176506.0,9
X40,804917.0,10
X40N,630428.0,9
OX40N,560024.0,10
XY25,491324.0,7
SR,260528.0,2
X5K,103628.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3712092.0,29
M-SC,1511565.0,17
H-MC,1233222.0,15
H-LC,1232610.0,15
M-LC,373819.0,4
M-MC,366073.0,2
L-SC,261187.0,3
L-MC,60335.0,1
L-LC,43255.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1216275.0      6
M-SC       XY24       845837.0      7
H-SC       AR         805219.0      5
           XR         789135.0      7
H-MC       XY24       571593.0      4
H-LC       X40        520475.0      5
M-MC       XY24       366073.0      2
H-SC       X40N       310509.0      4
           SR         260528.0      2
           OX40N      258164.0      4
H-LC       X40N       235584.0      3
H-MC       X40        211206.0      4
H-LC       AR         203575.0      2
H-MC       XY25       182392.0      2
L-SC       XR         172441.0      2
M-SC       AR         167712.0      2
M-LC       XY24       159832.0      2
M-SC       XY25       153278.0      1
           OX40N      141271.0      4
           XR         130231.0      2
H-LC       XY25       112399.0      3
M-LC       XR         110359.0      1
           X5K        103628.0      1
H-MC       XR          99452.0      1
H-LC       X200        88975.0      1
L-SC       OX40N       88746.0      1
H-MC       X40N        84335.0      2
M-SC       X40         73236.0      1
H-SC       MH          72262.0      1
H-MC       OX40N       71843.0      1
H-LC       XY24        71602.0      1
L-MC       XR          60335.0      1
L-LC       XY25        43255.0      1
H-MC       MH          12401.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
